In [14]:
%%writefile mapper.py

import sys
import re


def eprint(*args, **kwargs):
    print(*args, file=sys.stderr, **kwargs)

for line in sys.stdin:
    try:
        article_id, text = line.strip().split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        eprint("reporter:counter:Wiki stats,Total words,%d" % 1)
        print("%s\t%d" % (word.lower(), 1))

Overwriting mapper.py


In [15]:
%%writefile reducer.py

import sys

current_key = None
word_sum = 0

Overwriting reducer.py


In [16]:
%%writefile -a reducer.py

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            print("%s\t%d" % (current_key, word_sum))
        word_sum = 0
        current_key = key
    word_sum += count

if current_key:
    print("%s\t%d" % (current_key, word_sum))

Appending to reducer.py


In [17]:
! hdfs dfs -ls /data/

Found 2 items
drwxrwxrwx   - jovyan supergroup          0 2020-10-05 14:51 /data/twitter
drwxrwxrwx   - jovyan supergroup          0 2020-10-05 14:51 /data/wiki


In [18]:
%%bash

OUT_DIR="coursera_mr_task0"
NUM_REDUCERS=8  # Do not change it

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapreduce.job.name="Streaming Word Count" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,reducer.py \
    -mapper "python3 mapper.py" \
    -combiner "python3 reducer.py" \
    -reducer "python3 reducer.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null

hdfs dfs -cat ${OUT_DIR}/part-00000 | head

0%however	1
0&\mathrm{if	1
0(8)320-1234	1
0)).(1	2
0,03	1
0,1,...,n	1
0,1,0	1
0,1,\dots,n	1
0,5	1
0,50	1


rm: `coursera_mr_task0': No such file or directory
22/03/10 17:51:55 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
22/03/10 17:51:55 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
22/03/10 17:51:55 INFO mapred.FileInputFormat: Total input files to process : 1
22/03/10 17:51:55 INFO mapreduce.JobSubmitter: number of splits:2
22/03/10 17:51:55 INFO Configuration.deprecation: yarn.resourcemanager.system-metrics-publisher.enabled is deprecated. Instead, use yarn.system-metrics-publisher.enabled
22/03/10 17:51:55 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1646930071210_0004
22/03/10 17:51:55 INFO conf.Configuration: resource-types.xml not found
22/03/10 17:51:55 INFO resource.ResourceUtils: Unable to find 'resource-types.xml'.
22/03/10 17:51:55 INFO resource.ResourceUtils: Adding resource type - name = memory-mb, units = Mi, type = COUNTABLE
22/03/10 17:51:55 INFO resource.ResourceUtils: Adding resource type - name = vcores, units 